In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier
%matplotlib inline

# Lesson Example

In [2]:
artworks = pd.read_csv('https://media.githubusercontent.com/media/MuseumofModernArt/collection/master/Artworks.csv')

In [3]:
artworks.columns

Index(['Title', 'Artist', 'ConstituentID', 'ArtistBio', 'Nationality',
       'BeginDate', 'EndDate', 'Gender', 'Date', 'Medium', 'Dimensions',
       'CreditLine', 'AccessionNumber', 'Classification', 'Department',
       'DateAcquired', 'Cataloged', 'ObjectID', 'URL', 'ThumbnailURL',
       'Circumference (cm)', 'Depth (cm)', 'Diameter (cm)', 'Height (cm)',
       'Length (cm)', 'Weight (kg)', 'Width (cm)', 'Seat Height (cm)',
       'Duration (sec.)'],
      dtype='object')

In [4]:
# Select Columns.
artworks = artworks[['Artist', 'Nationality', 'Gender', 'Date', 'Department',
                    'DateAcquired', 'URL', 'ThumbnailURL', 'Height (cm)', 'Width (cm)']]

# Convert URL's to booleans.
artworks['URL'] = artworks['URL'].notnull()
artworks['ThumbnailURL'] = artworks['ThumbnailURL'].notnull()

# Drop films and some other tricky rows.
artworks = artworks[artworks['Department']!='Film']
artworks = artworks[artworks['Department']!='Media and Performance Art']
artworks = artworks[artworks['Department']!='Fluxus Collection']

# Drop missing data.
artworks = artworks.dropna()

In [5]:
artworks.head()

,Artist,Nationality,Gender,Date,Department,DateAcquired,URL,ThumbnailURL,Height (cm),Width (cm)
0,Otto Wagner,(Austrian),(Male),1896,Architecture & Design,1996-04-09,True,True,48.6000,168.9000
1,Christian de Portzamparc,(French),(Male),1987,Architecture & Design,1995-01-17,True,True,40.6401,29.8451
2,Emil Hoppe,(Austrian),(Male),1903,Architecture & Design,1997-01-15,True,True,34.3000,31.8000
3,Bernard Tschumi,(),(Male),1980,Architecture & Design,1995-01-17,True,True,50.8000,50.8000
4,Emil Hoppe,(Austrian),(Male),1903,Architecture & Design,1997-01-15,True,True,38.4000,19.1000


In [6]:
# Get data types.
artworks.dtypes

Artist           object
Nationality      object
Gender           object
Date             object
Department       object
DateAcquired     object
URL                bool
ThumbnailURL       bool
Height (cm)     float64
Width (cm)      float64
dtype: object

In [7]:
artworks['DateAcquired'] = pd.to_datetime(artworks.DateAcquired)
artworks['YearAcquired'] = artworks.DateAcquired.dt.year
artworks['YearAcquired'].dtype

dtype('int64')

In [8]:
# Remove multiple nationalities, genders, and artists.
artworks.loc[artworks['Gender'].str.contains('\) \('), 'Gender'] = '\(multiple_persons\)'
artworks.loc[artworks['Nationality'].str.contains('\) \('), 'Nationality'] = '\(multiple_nationalities\)'
artworks.loc[artworks['Artist'].str.contains(','), 'Artist'] = 'Multiple_Artists'

# Convert dates to start date, cutting down number of distinct examples.
artworks['Date'] = pd.Series(artworks.Date.str.extract(
    '([0-9]{4})', expand=False))[:-1]

# Final column drops and NA drop.
X = artworks.drop(['Department', 'DateAcquired', 'Artist', 'Nationality', 'Date'], 1)

# Create dummies separately.
artists = pd.get_dummies(artworks.Artist)
nationalities = pd.get_dummies(artworks.Nationality)
dates = pd.get_dummies(artworks.Date)

# Concat with other variables, but artists slows this wayyyyy down so we'll keep it out for now
X = pd.get_dummies(X, sparse=True)
X = pd.concat([X, nationalities, dates], axis=1)

Y = artworks.Department

In [10]:
# Alright! We've done our prep, let's build the model.
# Neural networks are hugely computationally intensive.
# This may take several minutes to run.

# Import the model.
from sklearn.neural_network import MLPClassifier
rows = 20000
# Establish and fit the model, with a single, 1000 perceptron layer.
mlp = MLPClassifier(hidden_layer_sizes=(1000,))
mlp.fit(X, Y)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(1000,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [11]:
mlp.score(X, Y)

0.7516103780969716

In [12]:
from sklearn.model_selection import cross_val_score
cross_val_score(mlp, X[:rows], Y[:rows], cv=5)

c:\python37\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
c:\python37\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


array([0.85453637, 0.83575   , 0.92575   , 0.924     , 0.84146037])

1-4 x 100, 200, 500, 1000

# 1 x 100

In [13]:
# random subset index
rows = np.random.choice(len(X), 10000, replace=False)

In [14]:
mlp = MLPClassifier(hidden_layer_sizes=(100,))
mlp.fit(X.iloc[rows], Y.iloc[rows])

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [15]:
mlp.score(X.iloc[rows], Y.iloc[rows])

0.663

In [16]:
cross_val_score(mlp, X.iloc[rows], Y.iloc[rows], cv=5)

array([0.48601399, 0.57871064, 0.637     , 0.5947974 , 0.25225225])

# 2 x 100

In [17]:
mlp = MLPClassifier(hidden_layer_sizes=(100, 100))
mlp.fit(X.iloc[rows], Y.iloc[rows])

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100, 100), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [18]:
mlp.score(X.iloc[rows], Y.iloc[rows])

0.5332

In [19]:
cross_val_score(mlp, X.iloc[rows], Y.iloc[rows], cv=5)

array([0.55094905, 0.57471264, 0.6125    , 0.53526763, 0.23523524])

# 3 x 100

In [20]:
mlp = MLPClassifier(hidden_layer_sizes=(100, 100, 100))
mlp.fit(X.iloc[rows], Y.iloc[rows])

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100, 100, 100), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [21]:
mlp.score(X.iloc[rows], Y.iloc[rows])

0.6404

In [22]:
cross_val_score(mlp, X.iloc[rows], Y.iloc[rows], cv=5)

array([0.56393606, 0.65517241, 0.6205    , 0.55327664, 0.52852853])

# 4 x 100

In [23]:
mlp = MLPClassifier(hidden_layer_sizes=(100, 100, 100, 100))
mlp.fit(X.iloc[rows], Y.iloc[rows])

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100, 100, 100, 100), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [24]:
mlp.score(X.iloc[rows], Y.iloc[rows])

0.6612

In [25]:
cross_val_score(mlp, X.iloc[rows], Y.iloc[rows], cv=5)

array([0.61688312, 0.64567716, 0.6145    , 0.55777889, 0.37187187])

# 1 x 200

In [26]:
mlp = MLPClassifier(hidden_layer_sizes=(200, ))
mlp.fit(X.iloc[rows], Y.iloc[rows])

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(200,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [27]:
mlp.score(X.iloc[rows], Y.iloc[rows])

0.3481

In [28]:
cross_val_score(mlp, X.iloc[rows], Y.iloc[rows], cv=5)

array([0.64535465, 0.60169915, 0.6095    , 0.23811906, 0.23323323])

# 2 x 200

In [29]:
mlp = MLPClassifier(hidden_layer_sizes=(200, 200 ))
mlp.fit(X.iloc[rows], Y.iloc[rows])

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(200, 200), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [30]:
mlp.score(X.iloc[rows], Y.iloc[rows])

0.5826

In [31]:
cross_val_score(mlp, X.iloc[rows], Y.iloc[rows], cv=5)

array([0.59290709, 0.48875562, 0.5365    , 0.23161581, 0.23523524])

# 3 x 200

In [32]:
mlp = MLPClassifier(hidden_layer_sizes=(200, 200, 200 ))
mlp.fit(X.iloc[rows], Y.iloc[rows])

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(200, 200, 200), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [33]:
mlp.score(X.iloc[rows], Y.iloc[rows])

0.6677

In [34]:
cross_val_score(mlp, X.iloc[rows], Y.iloc[rows], cv=5)

array([0.62187812, 0.58370815, 0.632     , 0.44822411, 0.42492492])

# 4 x 200

In [35]:
mlp = MLPClassifier(hidden_layer_sizes=(200, 200, 200, 200 ))
mlp.fit(X.iloc[rows], Y.iloc[rows])

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(200, 200, 200, 200), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [36]:
mlp.score(X.iloc[rows], Y.iloc[rows])

0.7036

In [37]:
cross_val_score(mlp, X.iloc[rows], Y.iloc[rows], cv=5)

c:\python37\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


array([0.66483516, 0.64667666, 0.615     , 0.47473737, 0.41291291])

# 1 x 500

In [38]:
mlp = MLPClassifier(hidden_layer_sizes=(500, ))
mlp.fit(X.iloc[rows], Y.iloc[rows])

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(500,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [39]:
mlp.score(X.iloc[rows], Y.iloc[rows])

0.6323

In [40]:
cross_val_score(mlp, X.iloc[rows], Y.iloc[rows], cv=5)

array([0.55294705, 0.59070465, 0.5415    , 0.23061531, 0.56956957])

# Final Notes

- 1x100
    - fit time = 9s
    - score = .66
    - cross val = poor
- 2x100
    - fit time = 8s
    - score = .64
    - cross val = poor
- 3x100
    - fit time = 18s
    - score = .53
    - cross val = mediocre, no seriously low scores
- 4x100
    - fit time = 33s
    - score = .66
    - cross val = mediocre, one low score
- 1x200
    - fit time = 9s
    - score = .35
    - cross val = poor, range from .23-.65
- 2x200
    - fit time = 15s
    - score = .58
    - cross val = poor, 2 seriously low scores
- 3x200
    - fit time = 46s
    - score = .67
    - cross val = mediocre, 2 scored in the .45ish range
- 4x200
    - fit time = 1m 55s
    - score = .70
    - cross val = mediocre, 2 scores in the .4-.5 range
- 1x500
    - fit time = 49s
    - score = .63
    - cross val = poor/mediocre, 1 very low scores
  